# Lookup Table Generation

Let's import the golden ratio package

In [19]:
!pip3 install import_ipynb
import import_ipynb
from golden_ratio import Goldice, INDETERMINATE, COMPLEX_INFINITY
from matplotlib import pyplot
import time
import pandas as pd
import math

Now, let's define some functions

In [40]:
def compute_closed_set(base : set):
    base_list = list(base)
    final_list = []
    for extern_element in base_list:
        for inner_element in base_list:
            x = extern_element + inner_element
            #print("Sum:", x)
            if ((not x in base) and (x != INDETERMINATE)):
                #print(x, "added")
                final_list.append(x)
            x = extern_element * inner_element
            #print("Product:", x)
            if ((not x in base) and (x != INDETERMINATE)):
                #print(x, "added")
                final_list.append(x)
    return final_list

def get_commonest(closed_set: list):
    commonest_map = dict()
    for elem in closed_set:
        if not elem in commonest_map:
            commonest_map[elem] = 0
        commonest_map[elem] = commonest_map[elem] + 1
    max_value = 0
    for elem in closed_set:
        if commonest_map[elem] > max_value:
            max_value = commonest_map[elem]
    return set([elem for elem in closed_set if commonest_map[elem] == max_value])

def compute_new_base(old_base: set):
    closed_set = compute_closed_set(old_base)
    commonest = get_commonest(closed_set)
    base = old_base.union(commonest)
    return base

def is_power_of_two(x):
    return (x & (x - 1)) == 0 and x != 0

def bits_needed(x):
    return len(bin(x).replace('0b', ''))

In [41]:
base = {Goldice(0,0), Goldice(1,0), Goldice(0,1)}
reciprocals = set()
for elem in base:
    reciprocals.add(elem.reciprocal())
print("Reciprocals:", reciprocals)
base = base.union(reciprocals)
print("Base + Reciprocals:", base)
negatives = set()
for elem in base:
    negatives.add(elem.opposite())
print("Negatives:", negatives)
base = base.union(negatives)
print("Base + Reciprocals + Negatives:", base)

execution_time = 0

base_length_list = []
min_range_a = []
max_range_a = []
min_range_a_bits = []
max_range_a_bits = []
min_range_b = []
max_range_b = []
min_range_b_bits = []
max_range_b_bits = []
execution_time_list = []

while len(base) != 128:
    start_time = time.time()
    base = compute_new_base(base)
    end_time = time.time()
    a = [elem.a for elem in base if elem != COMPLEX_INFINITY]
    b = [elem.b for elem in base if elem != COMPLEX_INFINITY]
    base_length_list.append(len(base))
    min_range_a.append(min(a))
    max_range_a.append(max(a))
    min_range_a_bits.append(bits_needed(min(a)))
    max_range_a_bits.append(bits_needed(max(a)))
    min_range_b.append(min(b))
    max_range_b.append(max(b))
    min_range_b_bits.append(bits_needed(min(b)))
    max_range_b_bits.append(bits_needed(max(b)))
    range_a = min(a), max(a), len(bin(min(a))), len(bin(max(a)))
    range_b = min(b), max(b), len(bin(min(b))), len(bin(max(b)))
    execution_time = execution_time + end_time - start_time
    execution_time_list.append(execution_time)
    print("-------------------------")
    print(len(base))
    print("Range of coefficient a:", range_a)
    print("Range of coefficient b:", range_b)
    print("Execution time: ", execution_time)


Reciprocals: {(1,0), (complex_infinity,complex_infinity), (-1,1)}
Base + Reciprocals: {(0,0), (1,0), (0,1), (complex_infinity,complex_infinity), (-1,1)}
Negatives: {(0,0), (complex_infinity,complex_infinity), (-1,0), (0,-1), (1,-1)}
Base + Reciprocals + Negatives: {(0,0), (1,0), (0,1), (complex_infinity,complex_infinity), (-1,0), (0,-1), (1,-1), (-1,1)}
-------------------------
12
Range of coefficient a: (-2, 2, 5, 4)
Range of coefficient b: (-1, 1, 4, 3)
Execution time:  0.00898599624633789
-------------------------
16
Range of coefficient a: (-3, 3, 5, 4)
Range of coefficient b: (-2, 2, 5, 4)
Execution time:  0.02040886878967285
-------------------------
20
Range of coefficient a: (-5, 5, 6, 5)
Range of coefficient b: (-3, 3, 5, 4)
Execution time:  0.031957149505615234
-------------------------
24
Range of coefficient a: (-8, 8, 7, 6)
Range of coefficient b: (-5, 5, 6, 5)
Execution time:  0.05594515800476074
-------------------------
28
Range of coefficient a: (-13, 13, 7, 6)
Range 

In [42]:
d = {'Base Length': base_length_list,'Min Value for a': min_range_a, 'Max Value for a': max_range_a, "Bits for min value of a": min_range_a_bits, "Bits for max value of a": max_range_a_bits, 'Min Value for b': min_range_b, 'Max Value for b': max_range_b, "Bits for min value of b": min_range_b_bits, "Bits for max value of b": max_range_b_bits, "Execution time (s)": execution_time_list}
df = pd.DataFrame(d)
df = df.set_index('Base Length')
df

,Min Value for a,Max Value for a,Bits for min value of a,Bits for max value of a,Min Value for b,Max Value for b,Bits for min value of b,Bits for max value of b,Execution time (s)
Base Length,,,,,,,,,
12,-2,2,3,2,-1,1,2,1,0.008986
16,-3,3,3,2,-2,2,3,2,0.020409
20,-5,5,4,3,-3,3,3,2,0.031957
24,-8,8,5,4,-5,5,4,3,0.055945
28,-13,13,5,4,-8,8,5,4,0.098737
32,-21,21,6,5,-13,13,5,4,0.150287
36,-34,34,7,6,-21,21,6,5,0.411376
40,-55,55,7,6,-34,34,7,6,0.549675
44,-89,89,8,7,-55,55,7,6,0.617213
